- To be used for inference purpose 

    - Inference for momentum and pressure

In [ ]:
import tensorflow as tf  # version 2.6.0
import joblib #Version: 1.2.0
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error #scikit-learn              1.3.0 
import numpy
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 26})


def scale_3D_inference(data_test_features,normalizer_test):
    
    # Step: Reshape Test dataset to 2D
    samples_test, timesteps_test, features_test = data_test_features.shape
    data_test_features = data_test_features.reshape(-1, features_test) 
    
    # Step: Scale features in test dataset         
    norm_test_features=normalizer_test.transform(data_test_features)
        
    print(normalizer_test.data_max_)
    print(normalizer_test.data_min_)
    
    # Step: Reshape back to 3D
    norm_test_features = norm_test_features.reshape(samples_test, timesteps_test, features_test)
    
    return norm_test_features


    - Give path locations to test data

In [ ]:
momentum=True
pressure=False

In [ ]:
if momemtum==True
    dirFile='../Models/LSTM_HAM'  #ML model for momentum
 
    dirFile_result='../Results'
    # kpn.create_directory_if_not_exists(dirFile_result) 
     
    filename_4_scalingfunction = dirFile + "/Scaling.save"
    
elif pressure==True
    dirFile='../Models/LSTM_HAM_Pressure'
    # kpn.create_directory_if_not_exists(dirFile)
   
    dirFile_result='../Results_Pressure'
    # kpn.create_directory_if_not_exists(dirFile_result)  
    
    filename_4_scalingfunction = dirFile + "/Scaling.save"
else:
    print('Please choose either momentum or pressure as true')
    


In [ ]:
# Inference

#Load model from path's location (momentum or pressure)
model=tf.keras.models.load_model(dirFile+'/best_model.h5')

#Load unseen test dataset and scale it for inference.  
    # CHECK : Test FEATURES data shape: num_samples (either 1 or all samples) x 3 x num_modes (num_modes=18 for momentum and 10 for pressure). 
    # Load scaling function for either momentum and pressure
    
test_data # Load features dataset

normalizer=joblib.load(filename_4_scalingfunction)

norm_test_features=scale_3D_inference(test_data,normalizer) # data_test_features
      

#Predict the residual using the model
addcorrection=model.predict(norm_test_features) #addcorrection shape : num_samples x num_modes


# #Plot the residual

for i in range(addcorrection.shape[1]):
    plt.figure(figsize=(20,12))
   
    plt.plot(addcorrection[:,i],label='ML predicted')
    
    plt.title('Comparison plot')
    
    plt.legend()
    
    filename = f'Comparison plot_{i}.png'
    
    plt.savefig(os.path.abspath(dirFile_result+'/'+filename), dpi = 200)
    
    plt.show()  

